In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [6]:
from backend.assistant.iterative_assistant import (
    IterativeAssistant,
    QueryGraph,
    AssistantSubject,
    AssistantLink,
    AssistantSubQuery,
    AssistantSubQueryType,
)

from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.ontology import OntologyManager, OntologyConfig, Graph
from backend.explorative.explorative_support import GuidanceManager
from backend.explorative.llm_query import (
    EnrichedEntitiesRelations,
    LLMQuery,
    QueryProgress,
)
from tqdm import tqdm
import pandas as pd

In [7]:
store = SPARQLStore(
    "http://localhost:7012/",
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
topic_man = GuidanceManager(ontology_manager)

assistant = IterativeAssistant(topic_man)

In [8]:
graph = QueryGraph(
    subjects=[
        AssistantSubject(
            internal_id="1",
            subject_id="Person",
        ),
        
        AssistantSubject(
            internal_id="2",
            subject_id="Place",
        )
    ],
    links=[
        AssistantLink(
            from_id="Person",
            to_id="Place",
            link_id="birthPlace",
            
            from_internal_id="1",
            to_internal_id="2",
        ),
    ],
)

In [32]:
resulting_ops = assistant.run_query("add the author link from a work the person", graph)

Llama.generate: 163 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     420.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   233 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   16065.69 ms /   234 tokens


Initial Ops operations=[Operation(operation=<OperationType.ADD: 'add'>, data=AssistantSubject(type='subject', subject_id='Author', internal_id='3', subqueries=[AssistantSubQuery(type='subquery', constraint_type=<AssistantSubQueryType.SUBJECT: 'subject'>, field='author', from_internal_id='1', from_id='Person'), AssistantSubQuery(type='subquery', constraint_type=<AssistantSubQueryType.SUBJECT: 'subject'>, field='author', from_internal_id='2', from_id='Place')], x=0.0, y=0.0)), Operation(operation=<OperationType.ADD: 'add'>, data=AssistantLink(type='link', from_id='Work', to_id='Author', from_internal_id='4', to_internal_id='3', link_id='author'))]
Candidate Ops operations=[Operation(operation=<OperationType.ADD: 'add'>, data=AssistantSubject(type='subject', subject_id='writer', internal_id='3', subqueries=[], x=0.0, y=0.0)), Operation(operation=<OperationType.ADD: 'add'>, data=AssistantSubject(type='subject', subject_id='novel', internal_id='3', subqueries=[], x=0.0, y=0.0)), Operation(o

Llama.generate: 163 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     420.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   184 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   12800.41 ms /   185 tokens


Constrained Ops operations=[ConstrainedOperation(operation=<OperationType.ADD: 'add'>, data=ConstrainedAssistantSubject(type='subject', subject_id=<SubjectID.writer: 'writer'>, internal_id='3', subqueries=[AssistantSubQuery(type='subquery', constraint_type=<AssistantSubQueryType.SUBJECT: 'subject'>, field='author', from_internal_id='1', from_id='Person')], x=0.0, y=0.0)), ConstrainedOperation(operation=<OperationType.ADD: 'add'>, data=ConstrainedAssistantLink(type='link', from_internal_id='1', to_internal_id='3', link_id=<LinkID.written work_author of preface_person: 'written work|author of preface|person'>))]
type='link' from_internal_id='1' to_internal_id='3' link_id=<LinkID.written work_author of preface_person: 'written work|author of preface|person'>
Constrained Ops operations=[Operation(operation=<OperationType.ADD: 'add'>, data=AssistantSubject(type='subject', subject_id='writer', internal_id='3', subqueries=[], x=0.0, y=0.0)), Operation(operation=<OperationType.ADD: 'add'>, dat

In [15]:
from llama_cpp_agent.gbnf_grammar_generator.gbnf_grammar_from_pydantic_models import (
    generate_gbnf_grammar_from_pydantic_models,
)
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Literal
from enum import Enum

class TestEnum(str, Enum):
    TEST = "test"
    TARGET = "target"
class TestType(BaseModel):
    type: Literal["test"]
    name: str


class TargetType(BaseModel):
    type: Literal["target"]
    other: str


class CombinerType(BaseModel):
    combined_type: list[TestType | TargetType]
    enum_literal: Literal[TestEnum.TEST]
    int_literal: Literal[1,5]

grammar = generate_gbnf_grammar_from_pydantic_models(
    [CombinerType],
)
print(grammar)

root ::= (" "| "\n") grammar-models
grammar-models ::= combiner-type
combiner-type ::= "{"  ws "\"combined_type\"" ": " combiner-type-combined-type ","  ws "\"enum_literal\"" ": " ("\"test\"") ","  ws "\"int_literal\"" ": " ("1"|"5")  ws "}"
test-type ::= "{"  ws "\"type\"" ": " ("\"test\"") ","  ws "\"name\"" ": " string  ws "}"
target-type ::= "{"  ws "\"type\"" ": " ("\"target\"") ","  ws "\"other\"" ": " string  ws "}"
combiner-type-combined-type-element-union ::= test-type | target-type
combiner-type-combined-type ::= "[" ws (combiner-type-combined-type-element-union)? ("," ws combiner-type-combined-type-element-union)* ws "]" 

boolean ::= "true" | "false"
null ::= "null"
string ::= "\"" (
        [^"\\] |
        "\\" (["\\/bfnrt] | "u" [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F])
      )* "\""
ws ::= ([ \t\n]+)
number ::= "-"? ([0-9]+ | [0-9]+ "." [0-9]+) ([eE] [-+]? [0-9]+)?


In [16]:

from llama_cpp.llama import Llama, LlamaGrammar
grammar_constrained = LlamaGrammar.from_string(grammar)
# grammar_constrained._grammar

In [18]:

response = topic_man.llama_model.create_chat_completion(
    grammar=grammar_constrained,
    messages=[
        {
            "role": "user",
            "content": "set the int literal 5 and enum literal test, create target",
        },
    ],
    max_tokens=128,
    temperature=0.1,
    seed=2
)
print(response["choices"][0]["message"]["content"])

Llama.generate: 8 prefix-match hit, remaining 13 prompt tokens to eval
llama_perf_context_print:        load time =     420.64 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    13 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    48 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3386.94 ms /    61 tokens


 {
    "combined_type": [
      {
        "type": "target",
        "other": "test"
      }
    ],
    "enum_literal": "test",
    "int_literal": 5
  }


In [22]:
CombinerType.model_validate_json(response["choices"][0]["message"]["content"])

CombinerType(combined_type=[TargetType(type='target', other='test')], enum_literal=<TestEnum.TEST: 'test'>, int_literal=5)